# FFT(Fast Fourier Transform)를 이용한 고속 다항식 곱셈 보고서

## 목차
- I. 서론: 다항식 곱셈의 계산 복잡도 분석  
- II. 다항식 곱셈의 구조  
  - II-1. 다항식의 두 가지 표현법  
  - II-2. 짝수홀수 분할과 재귀 구조  
- III. DFT, FFT, IFFT의 정의 및 수식  
  - III-1. DFT, FFT, IFFT의 개념  
  - III-2. DFT가 성립하기 위한 조건  
  - III-3. FFT와 IFFT가 동일한 이유  
- IV. 단위근  
  - IV-1. primitive 단위근  
  - IV-2. 단위근의 대칭성  
  - IV-3. even/odd($x^2$)에서 보존되는 단위근의 재귀적 성질  
- V. 구현
  - V-1. Cooley–Tukey 알고리즘
  - V-2. 파이썬 코드 

---

## I. 서론: 다항식 곱셈의 계산 복잡도 분석

두 다항식 $A(x), B(x)$의 곱 $C(x) = A(x)B(x)$를 직접 계산하면 $\mathcal{O}(n^2)$의 복잡도를 갖는다.  
그러나 FFT(고속 푸리에 변환)를 이용하면 $\mathcal{O}(n \log n)$의 시간에 계산이 가능하다.  
이는 다음의 변환 과정으로 가능하다:
#### FFT,IFFT를 적용한 다항식 곱셈 흐름도:
![fft](../image/fft.png)
1. $A(x), B(x)$를 점값 표현으로 변환 → FFT  
2. 점값에서 대응 항끼리 곱함  
$$
C(x_i) = A(x_i) \times B(x_i)
$$
3. 다시 계수 표현으로 복원 → IFFT  

---

## II. 다항식 곱셈의 구조

### II-1) 다항식의 두 가지 표현법

- **계수 표현 (Coefficient form)**  
  $A(x) = a_0 + a_1x + a_2x^2 + \cdots + a_{n-1}x^{n-1}$

- **점값 표현 (Point-value form)**  
  $\{ (x_0, A(x_0)), (x_1, A(x_1)), \dots, (x_{n-1}, A(x_{n-1})) \}$

> $n$개의 서로 다른 점에서의 값이 주어지면 $n-1$차 다항식을 유일하게 복원할 수 있다.

## 점값 표현의 유일성 증명

> **$d+1$개의 서로 다른 점을 모두 지나는 $d$차 다항식은 오직 하나만 존재한다.**

이 성질은 FFT를 기반으로 한 다항식 곱셈에서 핵심이 된다.

![d+1](../image/d+1.png)
위와 같이 4개의 점이 주어졌을때 이 점을 모두 통과하는 3차 이하의 다항식은 하나만 존재한다. 이 성질은 차수가 3일 때 뿐 아니라 항상 적용되며 증명은 다음과 같다:

$d$차 다항식 $P(x)$와 $Q(x)$가 서로 다른 $d+1$개의 점 $(x_0, y_0), (x_1, y_1), \dots, (x_d, y_d)$을 모두 지난다고 가정하자.

즉, 모든 $i=0$부터 $d$까지에 대해 다음이 성립한다:

$$
P(x_i) = y_i,\quad Q(x_i) = y_i
$$

두 다항식의 차를 $R(x) = P(x) - Q(x)$라고 정의하자.

그러면 $R(x)$는 $d$차 이하의 다항식이면서 다음을 만족한다:

$$
R(x_i) = 0 \quad \text{for all } i = 0, 1, \dots, d
$$

즉, $R(x)$는 서로 다른 $d+1$개의 점에서 0이 된다.

그러나 $d$차 이하의 다항식은 **최대 $d$개의 서로 다른 근**만 가질 수 있으므로, $R(x)$가 $d+1$개의 서로 다른 점에서 0이 되는 것은 **불가능**하다. (단, $R(x) \equiv 0$인 경우 제외)

따라서 $R(x) \equiv 0$, 즉 $P(x) = Q(x)$여야 한다.

**결론:** 서로 다른 $d+1$개의 점을 모두 지나는 $d$차 다항식은 유일하다.

---

### II-2) 짝수/홀수 분할과 재귀 구조

## 짝수, 홀수 함수의 대칭성 
![even vs odd](../image/even_odd.png)
다항식 $P(x)$가 계수 $a_i$를 가질 때, 짝수 차수 항과 홀수 차수 항을 분리하면 함수의 대칭성을 활용한 구조로 나타낼 수 있다. 
$$
P(x) = \sum_{i=0}^{n-1} a_i x^i = a_0 + a_1 x + a_2 x^2 + \cdots + a_{n-1} x^{n-1}
$$

이때, 짝수 차수 항들로 구성된 다항식과 홀수 차수 항들로 구성된 다항식은 다음과 같이 정의된다:

$$
P_{\text{even}}(x) = a_0 + a_2 x + a_4 x^2 + \cdots = \sum_{j=0}^{\frac{n}{2}-1} a_{2j} x^j
$$

$$
P_{\text{odd}}(x) = a_1 + a_3 x + a_5 x^2 + \cdots = \sum_{j=0}^{\frac{n}{2}-1} a_{2j+1} x^j
$$

따라서 원래의 다항식 $P(x)$는 다음과 같이 분해된다:

$$
P(x) = P_{\text{even}}(x^2) + x \cdot P_{\text{odd}}(x^2)
$$

이 구조는 FFT의 재귀 분할 구조를 형성한다.  
특히, $P(x)$를 $P_{\text{even}}(x^2)$와 $x \cdot P_{\text{odd}}(x^2)$로 분리함으로써, DFT 계산을 절반 크기의 부분 문제로 나눌 수 있다.

예를 들어, $n$차 단위복소근 $\omega$를 사용하여 $P(x)$를 DFT로 계산할 때, 평가점이 $\omega^k$라고 하면 다음과 같이 계산된다:

$$
\text{DFT}_n[P](\omega^k) = P(\omega^k) = P_{\text{even}}((\omega^k)^2) + \omega^k \cdot P_{\text{odd}}((\omega^k)^2)
$$

여기서 $(\omega^k)^2 = \omega^{2k}$이므로, 위 식은 $\omega^2$를 새로운 단위복소근으로 갖는 구조로 해석할 수 있다.  
즉, $\omega^2$는 $n$이 짝수일 때 $\frac{n}{2}$차 단위복소근이 되며,  
이는 곧 **$P_{\text{even}}$과 $P_{\text{odd}}$ 각각에 대해 $n/2$개의 점에서 DFT를 수행하는 것과 같다.**

- $P_{\text{even}}$을 $\omega^{2k}$에 대해 평가 → $n/2$개의 점에서의 DFT
- $P_{\text{odd}}$도 $\omega^{2k}$에 대해 평가 → $n/2$개의 점에서의 DFT

결과적으로, $P(x)$에 대한 $n$개의 DFT 값을 구하려면 두 개의 절반 크기 DFT만 계산하면 된다.  
이 과정을 $\log_2 n$ 단계만큼 반복함으로써 전체 계산량을 $O(n \log n)$으로 줄일 수 있다.


#### 재귀 흐름도:
![recursive](../image/recurse.png)
- 입력 다항식 $P(x)$  
- 짝수 차항: $P_{\text{even}}(x^2)$, 홀수 차항: $P_{\text{odd}}(x^2)$  
- 재귀적으로 FFT 수행  
- 점값 계산: $P_{\text{even}}(w^2)$, $P_{\text{odd}}(w^2)$  
- 병합: $P(w) = P_{\text{even}}(w^2) + w \cdot P_{\text{odd}}(w^2)$

---

#### 단위근의 필요성

##### $n$ 깊이 재귀에서 pair의 보존 필요 → 실수로는 불가능

- 제곱했을 때 원점 대칭이 보존되어야함
- $n = 8$일 때, 재귀를 통해 $n/2$, $n/4$ 등으로 나누어 계산
- 각 단계에서 분할된 even/odd를 합칠 때 복소수 회전의 특성이 필요
- 실수만으로는 $w_n^k$의 회전 성질을 보존할 수 없음

---

## III. DFT, FFT, IFFT의 정의 및 수식

### III-1.DFT, FFT, IFFT의 개념

#### DFT(Discrete Fourier Transform) (이산 푸리에 변환)
- 주어진 길이 $n$의 복소수열 $\{a_0, a_1, \dots, a_{n-1}\}$를 복소수 주파수 성분으로 분해하는 선형 변환
- 다항식을 점값 표현으로 변환하여 $A(w_n^k)$ 꼴로 표현 가능하게 함

#### FFT(Fast Fourier Transform) (고속 푸리에 변환)
- DFT를 $\mathcal{O}(n^2)$ 대신 $\mathcal{O}(n \log n)$ 시간에 수행하는 재귀적 알고리즘

#### IFFT(Inverse Fast Fourier Transform) (역 이산 푸리에 변환)
- DFT의 역변환, 점값 표현을 계수 표현으로 복원


### III-2. DFT가 성립하기 위한 조건

길이가 $n$인 복소수 벡터 $\mathbf{a} = (a_0, a_1, \dots, a_{n-1})$에 대해, DFT는 다음과 같은 수식을 따른다.

$$
A_k = \sum_{j=0}^{n-1} a_j \cdot \omega_n^{jk} \quad (0 \leq k < n)
$$

여기서 $\omega_n = e^{-2\pi i / n}$은 $n$번째 **단위근**이다.  
이때 $\omega_n^n = 1$을 만족하고, $\omega_n^k \neq 1$ for $0 < k < n$ 인 경우를 **primitive**한 단위근이라고 한다.

> **DFT가 성립하기 위한 조건**
> - 변환에 사용되는 $\omega_n$은 $n$번째 단위근이어야 한다.
> - 인덱스 $jk$는 모듈로 $n$에 따라 해석되므로 **인덱스 정렬과 순서**에 유의해야 한다.

> **단위근의 의미**  
> 복소평면 상에서 $\omega_n = e^{-2\pi i / n}$는 단위원의 원을 $n$등분하는 각도를 의미한다.  
> 즉, $\omega_n^k$는 반지름 1인 원에서 $k$번째 점을 가리키며, 이는 주기적 회전의 성질을 내포한다.

---
### III-3. FFT와 IFFT가 동일한 이유  

FFT는 다음과 같이 다항식을 점값으로 변환하는 DFT 행렬 곱 형태로 표현할 수 있다:

$P(x) = p_0 + p_1x + p_2x^2 + \cdots + p_{n-1}x^{n-1}$ 에 대해,

$$
\begin{bmatrix}
P(w^0) \\
P(w^1) \\
P(w^2) \\
\vdots \\
P(w^{n-1})
\end{bmatrix}
=
\begin{bmatrix}
1 & 1 & 1 & \cdots & 1 \\
1 & w & w^2 & \cdots & w^{n-1} \\
1 & w^2 & w^4 & \cdots & w^{2(n-1)} \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
1 & w^{n-1} & w^{2(n-1)} & \cdots & w^{(n-1)^2}
\end{bmatrix}
\begin{bmatrix}
p_0 \\
p_1 \\
p_2 \\
\vdots \\
p_{n-1}
\end{bmatrix}
$$

여기서 $w = e^{\frac{2\pi i}{n}}$ 는 $n$번째 단위근이다. 위 행렬은 DFT 행렬 $\mathbf{F}_n$이라 부르며, 이를 통해 점값 표현으로의 변환이 이루어진다.

---

이제 IFFT는 이 DFT 행렬의 역행렬을 통해 계수로 되돌리는 연산이다:

$$
\begin{bmatrix}
p_0 \\
p_1 \\
p_2 \\
\vdots \\
p_{n-1}
\end{bmatrix}
=
\mathbf{F}_n^{-1}
\begin{bmatrix}
P(w^0) \\
P(w^1) \\
P(w^2) \\
\vdots \\
P(w^{n-1})
\end{bmatrix}
$$

그리고 $\mathbf{F}_n^{-1}$은 다음과 같이 나타낼 수 있다:

$$
\mathbf{F}_n^{-1} = \frac{1}{n}
\begin{bmatrix}
1 & 1 & 1 & \cdots & 1 \\
1 & w^{-1} & w^{-2} & \cdots & w^{-(n-1)} \\
1 & w^{-2} & w^{-4} & \cdots & w^{-2(n-1)} \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
1 & w^{-(n-1)} & w^{-2(n-1)} & \cdots & w^{-(n-1)^2}
\end{bmatrix}
$$

즉, IFFT 역시 DFT 구조를 유지한 채, 단위근을 켤레 복소수로 바꾸고 전체에 $\frac{1}{n}$을 곱하는 방식으로 계산된다. 이는 DFT의 **conjugate transpose가 역행렬**이 됨을 의미한다.

---

## IV. 단위근
### 단위근의 특성:
![w^k](../image/w^k.png)
### IV-1) primitive 단위근
- $w_n$이 primitive root가 되려면:
  - $w_n^n = 1$
  - $w_n^k \neq 1$ for $0 < k < n$

→ $n$개의 고유한 회전 각도를 제공하며, $n$차 다항식을 고르게 분해 가능

### IV-2) 단위근의 대칭성
- Figure 2: 단위근 $w_n^k = e^{-\frac{2\pi i k}{n}}$는 복소평면의 단위원 상에서 $2\pi k/n$ 만큼 균등하게 회전하며 배치된다.
- 앞의 절반(즉, $k = 0, 1, \dots, \frac{n}{2}-1$) 단위근에 마이너스를 붙인 값들이 뒤의 절반($k = \frac{n}{2}, \frac{n}{2}+1, \dots, n-1$) 단위근과 정확히 일치한다.

$$
\begin{aligned}
-w_n^k &= -e^{-\frac{2\pi i k}{n}} \\
&= e^{i \pi} \cdot e^{-\frac{2\pi i k}{n}} \quad \text{(since } -1 = e^{i \pi} \text{)} \\
&= e^{-\frac{2\pi i k}{n} + i \pi} \\
&= e^{-\frac{2\pi i k}{n} + \frac{2\pi i n}{2n}} \quad \text{(rewrite } i \pi = \frac{2\pi i n}{2n} \text{)} \\
&= e^{-\frac{2\pi i k}{n} + \frac{2\pi i (n/2)}{n}} \\
&= e^{-\frac{2\pi i}{n}(k - \frac{n}{2})} \\
&= w_n^{k + \frac{n}{2}} \quad \text{(since } w_n^{m} = e^{-\frac{2\pi i m}{n}} \text{)}
\end{aligned}
$$

- 따라서,

$$
-w_n^k = w_n^{k + \frac{n}{2}} \quad \text{for } 0 \leq k < \frac{n}{2}
$$

- 이 결과는 FFT에서 짝수 인덱스와 홀수 인덱스의 분할 후 재조합할 때, 절반 길이 단위근들만으로 전체 단위근 집합을 표현할 수 있음을 의미한다.

- 즉, 다음 두 식은 대칭적이고 효율적으로 계산될 수 있다.
  $$
  P(w_n^k) = P_{\text{even}}(w_n^{2k}) + w_n^k P_{\text{odd}}(w_n^{2k})
  $$
  $$
  P(w_n^{k + \frac{n}{2}}) = P_{\text{even}}(w_n^{2k}) - w_n^k P_{\text{odd}}(w_n^{2k})
  $$

- 이로 인해 단위근의 절반만 사용해도 전체 계산이 가능해지며, FFT는 재귀 단계마다 계산량을 절반으로 줄인다.

### IV-3) even/odd($x^2$)에서 보존되는 단위근의 재귀적 성질
#### 단위원 상에서 $w_n^k$들이 균등하게 회전하여 배치되는 모습
![roots of unity](../image/unity.png)
앞서 재귀 분할 시,

$P(x) = P_{\text{even}}(x^2) + x \cdot P_{\text{odd}}(x^2)$

여기서 $n$번째 단위근 $\omega_n$이 있을 때, $\omega_n^2$는 $\omega_{n/2}$에 해당한다.

왜냐하면:

$$
\omega_n = e^{\frac{2\pi i}{n}} \Rightarrow (\omega_n)^2 = e^{\frac{4\pi i}{n}} = e^{\frac{2\pi i}{n/2}} = \omega_{n/2}
$$

즉, 제곱 연산은 **각도를 2배**로 만들고, **분할 수는 절반**으로 줄어든다.  
이로 인해 $n$개의 단위근 중에서 **짝수 인덱스 단위근들**만으로 $n/2$개의 단위근이 재귀적으로 구성된다.


---

## V. 구현

```python
# FFT 구현

import cmath 

def fft(array, input_solution):
    n = len(array)
    if n == 1:
        return array
    
    array2 = [array[2 * i] for i in range(n // 2)]
    array3 = [array[2 * i + 1] for i in range(n // 2)]

    squared_half_solution = [input_solution[i] ** 2 for i in range(n // 2)]

    P = fft(array2, squared_half_solution)
    Q = fft(array3, squared_half_solution)

    result = [0] * n
    for i in range(n // 2):
        result[i] = P[i] + input_solution[i] * Q[i]
        result[i + n // 2] = P[i] - input_solution[i] * Q[i]
    
    return result

array = [1, 2, 3, 4, 5, 6, 7, 8]

n = len(array)
w_list = [cmath.exp(2j * cmath.pi * i / n) for i in range(n)]

result = fft(array, w_list[:n // 2])
print(result)
```

In [ ]:
#정확한 수식 구현

from sympy import symbols, exp, I, pi, Rational, simplify

def fft(array, input_solution):
    n = len(array)
    if n == 1:
        return array
    
    result = []
    array2 = [array[2*i] for i in range(n//2)]
    array3 = [array[2*i+1] for i in range(n//2)]

    squared_half_solution = [simplify(w**2) for w in input_solution[:n//2]]

    P = fft(array2, squared_half_solution)
    Q = fft(array3, squared_half_solution)

    for i in range(n//2):
        result.append(simplify(P[i] + input_solution[i] * Q[i]))
    for i in range(n//2):
        result.append(simplify(P[i] - input_solution[i] * Q[i]))

    return result

# 입력
n = 8
w_list = [exp(2 * pi * I * i / n) for i in range(n)]
array = [Rational(x) for x in [1,2,3,4,5,6,7,8]]

# 실행
result = fft(array, w_list[:n//2])
for val in result:
    print(val)